## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Michel Gonzalez, Group: 11

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
#!pip install scikit-learn==1.0.2

In [2]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [3]:
df_X, s_y_ordered = load_diabetes(return_X_y=True, as_frame=True)

# adds the intercept column for later use in beta_hat, and also shuffles the data 
# for better traning of our model
df_X = pd.concat([pd.DataFrame({'Intercept' : np.ones(len(df_X))}), df_X], axis = 1)

# Shuffles our data and reorders the 
# target data by using the new index order
df_X = df_X.sample(frac = 1)

new_idx = df_X.index

s_y = s_y_ordered.copy(deep = True)

for i in range(0, len(s_y)):

  s_y[i] = s_y_ordered[new_idx[i]]

len(df_X), len(s_y)

(442, 442)

## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [4]:
def get_linear_regression_model( df_X, s_y ):
    # your code here

    #beta_hat = (X * XT)^-1 * XT * y
    # uses linear lstsquares method to solve for beta_hat
    beta_hat, residuals, rank, s = np.linalg.lstsq(df_X, s_y, rcond = None )

    return pd.Series(beta_hat)

In [5]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
print(beta_hat)

0    3.682765
1    3.484999
2    0.001342
3    2.192263
dtype: float64


In [6]:
def predict_linear_regression_value( beta_hat, df_X ):
    # your code here

    # Multiplies the data set by the beta_hat that was created
    # and creates a series s_y_hat with predicted values
    s_y_hat = np.matmul(df_X, beta_hat)

    return np.array(s_y_hat)

In [7]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

array([2.27838225, 5.10744585, 2.45066005])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [8]:
def partition_data( df_X, s_y, k):
    # your code here

    # Creates the dictionaries that will hold each fold and series of data
    # it is assumed that df_X was properly shuffled before partitioning
    dict_k_df_X = {}

    dict_k_s_y = {}

    # partitions the data set objects by using a random number selcted from 1 through k inclusive
    # then it creates each dictionary
    random_indx = np.random.randint(1, k + 1, size = len(s_y))

    for i in range (1, k + 1):

        dict_k_df_X[i] = df_X[random_indx == i]

        dict_k_s_y[i] = np.array(s_y[random_indx == i])

    return dict_k_df_X, dict_k_s_y

In [9]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [10]:
# Check fold sizes

total = 0

for i in range (1, 6):
  
  print('Fold ' + str(i) + ' length of dataframe is ' + str(len(dict_k_df_X[i])) + ' and length of series is ' + str(len(dict_k_s_y[i])))

  total += len(dict_k_df_X[i])

print('The sum of the number of elements in each fold is ' + str(total) + ' and there are ' + str(len(df_X)) + ' rows in the original df')

Fold 1 length of dataframe is 91 and length of series is 91
Fold 2 length of dataframe is 98 and length of series is 98
Fold 3 length of dataframe is 77 and length of series is 77
Fold 4 length of dataframe is 75 and length of series is 75
Fold 5 length of dataframe is 101 and length of series is 101
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [11]:
def get_mae(s_y, s_y_hat):
    # your code here

    # mae = sum((s_y[i] - s_y_hat[i])/n)
    mae = 0

    # This will get the mean absolute error by using the actual values and
    # the predicted values
    n = len(s_y)

    for i in range(0, n):

      mae += abs((s_y[i] - s_y_hat[i]))/n

    return float(mae)

In [12]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [13]:
# Creates an array that will hold the MAE vlaues 
mae = np.array([])

# This will access all the information in each fold [1, k]
for k in dict_k_df_X.keys():
    
    # your code here

    # Extracts the test data that will be used at the 
    # end of the regression
    test_X_k = dict_k_df_X[k]

    test_s_y_k = dict_k_s_y[k]

    # Creates empty shells that will hold the training data
    training_X_k = pd.DataFrame(columns = df_X.columns)

    training_s_y_k = np.array([])

    # compiles all the data in the other folds into one so it can be used to 
    # train the model
    for i in dict_k_df_X.keys():

      if(i != k):
        
        training_X_k = training_X_k.append(dict_k_df_X[i])

        training_s_y_k = np.append(np.array(training_s_y_k), np.array(dict_k_s_y[i]))

      else:

        continue
    
    # Creates the beta vetcor to find for predicttions is y_hat = df_X * beta_hat
    beta_hat_k = get_linear_regression_model(training_X_k, training_s_y_k)

    # Creates the predicted values vetcor
    s_y_hat_k = predict_linear_regression_value(beta_hat_k, test_X_k)

    # Finds the mean absolute error by using the expected values vs the actule values
    # for each fold
    mae_val = get_mae(test_s_y_k, s_y_hat_k)

    mae = np.append(mae, mae_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  


In [14]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 42.54, the max MAE is 45.79, and the mean MAE is 44.12


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [15]:
df_X, s_y_ordered = load_iris(return_X_y=True, as_frame=True)

# This will shuffle the data and reorder the target data
# by the new index
df_X = df_X.sample(frac = 1)

new_idx = df_X.index

s_y = s_y_ordered.copy(deep = True)

for i in range(0, len(s_y)):

  s_y[i] = s_y_ordered[new_idx[i]]

len(df_X), len(s_y)

(150, 150)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [16]:
# this will partiition our data into similar but still random lengths
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

total = 0

for i in range (1, 6):
  
  print('Fold ' + str(i) + ' length of dataframe is ' + str(len(dict_k_df_X[i])) + ' and length of series is ' + str(len(dict_k_s_y[i])))

  total += len(dict_k_df_X[i])

print('The sum of the number of elements in each fold is ' + str(total) + ' and there are ' + str(len(df_X)) + ' rows in the original df')

Fold 1 length of dataframe is 23 and length of series is 23
Fold 2 length of dataframe is 37 and length of series is 37
Fold 3 length of dataframe is 29 and length of series is 29
Fold 4 length of dataframe is 28 and length of series is 28
Fold 5 length of dataframe is 33 and length of series is 33
The sum of the number of elements in each fold is 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [17]:
def get_acc( s_1, s_2 ):
    # your code here

    # Gets the accuracy by using the boolean array outpu and 
    # counting all the true vales, then divide by the total
    # number of elements 
    n = len(s_1)

    acc_array = np.array(s_1 == s_2)

    count = 0

    for i in range(0, len(acc_array)):

      if(acc_array[i] == True):

        count += 1

      else:

        continue
    
    acc = count/n

    return acc

In [18]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [19]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])

# This will get the number of inner loop
# folds to get average
div = max(dict_k_df_X.keys()) - 1

for k in dict_k_df_X.keys():
  # your code here

  # Creates the test and empty traning sets
  test_k_df_X = dict_k_df_X[k]

  test_k_s_y = dict_k_s_y[k]

  training_k_df_X = pd.DataFrame(columns = df_X.columns)

  training_k_s_y = np.array([])

  # This is used to keep track of the accuracy
  # from each min impurity decrease
  acc = []

  for pos_min_impurity in possible_min_impurity_decrease:

    clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = pos_min_impurity)

    # Inner loop cross validation code here (use 4 folds, where the fold does not include k)

    avg = 0

    # Cycles through the 4 folds using one as the test for the modle
    # and the other 3 folds as the traning sets
    for j in dict_k_df_X.keys():

      if(j != k):

        test_j_df_X = dict_k_df_X[j]

        test_j_s_y = dict_k_s_y[j]

        training_j_df_X = pd.DataFrame(columns = df_X.columns)

        training_j_s_y = np.array([])

        # creates the inner traning sets for each test fold
        for i in dict_k_df_X.keys():

          if(i != k and i != j):

            training_j_df_X = training_j_df_X.append(dict_k_df_X[j])

            training_j_s_y = np.append(np.array(training_j_s_y), np.array(dict_k_s_y[j]))

          else:

            continue

        # Trains a tree and gets its accuracy    
        clf = clf.fit(training_j_df_X, training_j_s_y)

        s_y_hat_j = clf.predict(test_j_df_X)

        accuracy = get_acc(s_y_hat_j, test_j_s_y)

        avg += accuracy


      else:

        continue

    avg = avg/div

    acc.append(avg)

    print('Testing', pos_min_impurity, 'min impurity decrease')
    print('    Average accuracy over 4 folds is ' + str(format(avg, '2.4f')))
  
  # Use best min impurity decrease to train model

  # Finds best min impurity decrese by finding the first maximum accuracy
  # (this will pick the lower min impurity decrease in case of a tie aslong as the list of values is in accending order)
  max_val = max(acc)

  max_indx = acc.index(max_val)

  print()
  print('Best min impurity decrease is', possible_min_impurity_decrease[max_indx])
  print()

  # Makes the traning set for the outer loop folds
  for m in dict_k_df_X.keys():

    if(m == k):

      continue
    
    else: 

      training_k_df_X = training_k_df_X.append(dict_k_df_X[m])

      training_k_s_y = np.append(np.array(training_k_s_y), np.array(dict_k_s_y[m]))

  # outer accuracy calculation

  clf = tree.DecisionTreeClassifier(min_impurity_decrease = possible_min_impurity_decrease[max_indx])

  clf = clf.fit(training_k_df_X, training_k_s_y)

  s_y_hat_k = clf.predict(test_k_df_X)

  this_acc = get_acc(s_y_hat_k, test_k_s_y)

  outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9848
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.9470
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.8934
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.4038

Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9848
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.9470
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.8282
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.4137

Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.9848
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.9470
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.8282
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [20]:
print("The min accuracy is {:.2f}, the max accuracy is {:.2f}, and the mean accuracy is {:.2f}".format(outer_acc.min(),outer_acc.max(),outer_acc.mean()))

The min accuracy is 0.85, the max accuracy is 0.97, and the mean accuracy is 0.93
